In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_3/models/B5R2b5_AN_1FC_fold3_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(659, 25)
(659, 25)
Normal:  (431, 25)
Abnormal:  (228, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,53,53,22,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,224.0,146.0,593.0,514.0,0.227687,0.162817,0.602914,0.571713,AB01 P2 C022.JPG,AB01 P2 C022.JPG
1,1,1,54,54,22,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,200.0,141.0,589.0,501.0,0.201802,0.156841,0.592793,0.557769,AB01 P1 C022.JPG,AB01 P1 C022.JPG
2,2,2,55,55,22,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,138.0,155.0,635.0,632.0,0.142066,0.172778,0.653137,0.703187,AB01 P4-2 C022.JPG,AB01 P4-2 C022.JPG
3,3,3,56,56,22,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,145.0,141.0,693.0,675.0,0.146209,0.156841,0.698556,0.750996,AB01 P3-1 C022.JPG,AB01 P3-1 C022.JPG
4,4,4,57,57,22,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,229.0,142.0,659.0,632.0,0.228164,0.158833,0.655971,0.703187,AB01 P4-1 C022.JPG,AB01 P4-1 C022.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 659 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.13440365, -0.14468555,  0.31250685, ..., -0.09287201,
         0.03966072, -0.08877676],
       [-0.01270042,  0.0329515 ,  0.18254654, ...,  0.29158041,
         0.00234716,  0.0622978 ],
       [-0.0385704 , -0.15124142,  0.24303642, ...,  0.12629598,
         0.06581297, -0.05191081],
       ...,
       [ 0.95557326,  0.7758072 , -0.09033869, ...,  0.81544155,
        -0.08822763,  0.35621512],
       [ 0.33424264,  0.54633039, -0.16841124, ...,  0.57088441,
        -0.11309762,  0.47734314],
       [-0.07582538,  0.20459595, -0.17429049, ...,  0.2611987 ,
         0.01250216,  0.06978479]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.134404,-0.144686,0.312507,0.306689,-0.184435,-0.114442,0.413968,-0.046321,-0.196257,0.090994,...,-0.120966,-0.018571,-0.161421,-0.072288,-0.146168,-0.042029,-0.138615,-0.092872,0.039661,-0.088777
1,-0.012700,0.032952,0.182547,0.509859,0.072718,0.019095,0.119693,0.090049,-0.118088,0.052309,...,-0.035913,0.023338,-0.103116,-0.024186,-0.020636,-0.155874,-0.054903,0.291580,0.002347,0.062298
2,-0.038570,-0.151241,0.243036,0.097685,-0.145521,0.019203,0.266536,-0.061893,-0.096292,-0.097586,...,0.026043,0.107804,0.166301,0.079127,-0.190753,0.007619,-0.085045,0.126296,0.065813,-0.051911
3,0.187290,-0.142313,0.086688,0.409962,-0.062759,-0.118456,0.407124,-0.027963,-0.164117,-0.074116,...,-0.008768,0.071318,-0.074451,0.064769,-0.122318,-0.043063,-0.056670,0.026847,0.037121,-0.095752
4,0.267432,0.010359,-0.015964,0.104505,-0.079670,-0.133789,0.128284,-0.055415,-0.110511,-0.000221,...,0.088748,-0.067996,0.091181,-0.033402,-0.120107,-0.128799,-0.059852,0.287224,-0.112638,-0.005345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,0.207622,0.852683,-0.087222,-0.116097,0.547903,0.127279,-0.204764,0.259616,0.848720,-0.218076,...,0.465253,-0.167234,0.204027,0.544383,0.415752,-0.170556,0.248261,0.722306,-0.018277,0.239554
655,-0.039471,0.082157,0.043223,0.019640,-0.031398,-0.084560,-0.025782,0.156900,-0.097803,-0.156753,...,0.063299,-0.062985,-0.082964,-0.105285,-0.010250,-0.119668,0.023026,0.069177,-0.150265,-0.106801
656,0.955573,0.775807,-0.090339,0.253198,0.813851,0.851752,-0.181679,0.395475,0.624833,-0.212398,...,0.741683,-0.185425,0.296211,0.955677,1.151373,-0.173989,0.624950,0.815442,-0.088228,0.356215
657,0.334243,0.546330,-0.168411,-0.075700,0.713374,-0.001233,-0.134750,0.020285,0.512056,-0.082235,...,0.308178,-0.164187,0.216478,0.631496,0.568316,-0.147370,0.398612,0.570884,-0.113098,0.477343


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.134404,-0.144686,0.312507,0.306689,-0.184435,-0.114442,0.413968,-0.046321,-0.196257,0.090994,...,-0.146168,-0.042029,-0.138615,-0.092872,0.039661,-0.088777,Abnormal,P2,FP-A,AB01
1,-0.012700,0.032952,0.182547,0.509859,0.072718,0.019095,0.119693,0.090049,-0.118088,0.052309,...,-0.020636,-0.155874,-0.054903,0.291580,0.002347,0.062298,Abnormal,P1,FP-A,AB01
2,-0.038570,-0.151241,0.243036,0.097685,-0.145521,0.019203,0.266536,-0.061893,-0.096292,-0.097586,...,-0.190753,0.007619,-0.085045,0.126296,0.065813,-0.051911,Abnormal,P42,FP-B,AB01
3,0.187290,-0.142313,0.086688,0.409962,-0.062759,-0.118456,0.407124,-0.027963,-0.164117,-0.074116,...,-0.122318,-0.043063,-0.056670,0.026847,0.037121,-0.095752,Abnormal,P31,FP-B,AB01
4,0.267432,0.010359,-0.015964,0.104505,-0.079670,-0.133789,0.128284,-0.055415,-0.110511,-0.000221,...,-0.120107,-0.128799,-0.059852,0.287224,-0.112638,-0.005345,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,0.207622,0.852683,-0.087222,-0.116097,0.547903,0.127279,-0.204764,0.259616,0.848720,-0.218076,...,0.415752,-0.170556,0.248261,0.722306,-0.018277,0.239554,Normal,P32,FP-B,Normal
655,-0.039471,0.082157,0.043223,0.019640,-0.031398,-0.084560,-0.025782,0.156900,-0.097803,-0.156753,...,-0.010250,-0.119668,0.023026,0.069177,-0.150265,-0.106801,Normal,P42,FP-B,Normal
656,0.955573,0.775807,-0.090339,0.253198,0.813851,0.851752,-0.181679,0.395475,0.624833,-0.212398,...,1.151373,-0.173989,0.624950,0.815442,-0.088228,0.356215,Normal,P52,FP-C,Normal
657,0.334243,0.546330,-0.168411,-0.075700,0.713374,-0.001233,-0.134750,0.020285,0.512056,-0.082235,...,0.568316,-0.147370,0.398612,0.570884,-0.113098,0.477343,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold3_1.csv')